Kaggle initialization

In [1]:

import numpy as np 
import pandas as pd
import torch
from tqdm import tqdm
import sys

sys.path.append('../input/tensorflow-great-barrier-reef')

In [2]:
!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/yolov5-font/Arial.ttf /root/.config/Ultralytics/

In [3]:
import greatbarrierreef
env = greatbarrierreef.make_env()# initialize the environment
iter_test = env.iter_test()      # an iterator which loops over the test set and sample submission

In [4]:
model = torch.hub.load('../input/yolov5-lib-ds', 
                       'custom', 
                       path='../input/reef-baseline-fold12/l6_3600_uflip_vm5_f12_up/f1/best.pt',
                       source='local',
                       force_reload=True)  
model.conf = 0.01 # model confidence

YOLOv5 🚀 2022-1-18 torch 1.9.1+cpu CPU

Fusing layers... 
Model Summary: 280 layers, 12308200 parameters, 0 gradients
Adding AutoShape... 


In [5]:
for idx, (img, pred_df) in enumerate(tqdm(iter_test)):
    anno = ''
    r = model(img, size=10000, augment=True)
    predictions = []
    if r.pandas().xyxy[0].shape[0] == 0:
        anno = ''
    else:
        for idx, row in r.pandas().xyxy[0].iterrows():
            if row.confidence > 0.28:
                anno += '{} {} {} {} {} '.format(row.confidence, int(row.xmin), int(row.ymin), int(row.xmax-row.xmin), int(row.ymax-row.ymin))
                predictions.append('{:.2f} {} {} {} {}'.format(row.confidence, row.xmin, row.ymin, row.xmax-row.xmin, row.ymax-row.ymin))
    
    prediction_str = ' '.join(predictions)
    pred_df['annotations'] = prediction_str
    env.predict(pred_df)
    print('Prediction:', prediction_str)

0it [00:00, ?it/s]

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


1it [01:23, 83.39s/it]

Prediction: 


2it [02:45, 82.92s/it]

Prediction: 0.61 302.99285888671875 556.5386352539062 63.851593017578125 53.27777099609375


3it [04:08, 82.82s/it]

Prediction: 0.33 298.892578125 564.9966430664062 62.661590576171875 50.1756591796875


In [6]:
sub_df = pd.read_csv('submission.csv')
sub_df.head()

,index,annotations
0,0,NaN
1,1,0.61 302.99285888671875 556.5386352539062 63.8...
2,2,0.33 298.892578125 564.9966430664062 62.661590...
